# **Principal Component Analysis (PCA) - II**


**objectives:**

In this Jupyter Notebook we continue with **Principal Component Analysis (PCA)**. The general learning goal is twofold:

1. Deeper understanding of principal component scores and principal directions
2. Understand how much variation each principal component explains, and how to reduce the dimensionality based on this measure

> - [A] Importing
> - [B] Centering (demeaning) the variables in your data
> - [C] PCA on the 2 length variables in the Iris dataset
> - [D] Properties of the principal component scores
> - [E] PCA on the full Iris data set
> - [F] Explained variance by each principal component
> - [G] [G] Dimensionality reduction using PCA



---
## **[A] Importing**

We import the same packages and data as in the previous lecture:

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.decomposition import PCA
from sklearn import datasets

iris = datasets.load_iris()

# Load X, y, variable labels for the Iris data set
X = iris.data
y = iris.target
variable_labels = iris.feature_names

# Create a dataframe for X
X = pd.DataFrame(X, columns=variable_labels)
X_only_length = X.loc[:, ['sepal length (cm)', 'petal length (cm)']]

---
## **[B] Centering (demeaning) the variables in your data**

In the previous lecture we have seen that the PCA functionality in scikit-learn automatically centers (demeans) the variables in your data, such that each column has zero mean.


In [ ]:
X_only_length_centered = X_only_length - X_only_length.mean()
X_only_length_centered.mean().round(6)

A benefit of centering your data is that it facilitates interpretation.
- A **positive value** for an (observation, variable) pair indicates that this observation has an **above average** value
- A **negative value** for an (observation, variable) pair indicates that this observation has a **below average** value

For example, consider the first observation (index 0) after centering:

In [ ]:
X_only_length_centered.iloc[0]

Both values are negative, implying that the 1st observation in our data has a below average sepal length and petal length. We can manually confirm this:

In [ ]:
X_only_length.iloc[0]

Comparing that against the average values:

In [ ]:
X_only_length.mean()

Indeed this shows that both values are below average.

Note that centering your data does **not** change the variance in your variables: 

In [ ]:
X_only_length.var()

In [ ]:
X_only_length_centered.var()

Because working with centered data is so useful, and to avoid any confusion, we will **overwrite** our `X` and `X_only_length` variables to be centered as well in the rest of this notebook:

In [ ]:
X = X - X.mean()
X_only_length = X_only_length - X_only_length.mean()

---
## **[C] PCA on the 2 length variables in the Iris dataset**

We continue were we left of in the previous notebook, by running PCA on the dataset `X_only_length` that only contains the two length variables from the Iris dataset:

In [ ]:
pca_only_length = PCA()
pca_only_length = pca_only_length.fit(X_only_length)

The **principal component directions**:
- Each direction is expressed as a linear combination of the **original variables**


In [ ]:
pc_directions = pd.DataFrame(
    pca_only_length.components_,
    index=['PC1', 'PC2'],
    columns = X_only_length.columns
)
pc_directions

The **principal component scores**, for each of the 150 flowers in the Iris dataset:
- The principal component scores for each observation is now expressed in terms of the two **principal components**

In [ ]:
pc_scores = pd.DataFrame(
    pca_only_length.transform(X_only_length),
    columns=['PC1', 'PC2'],
)
pc_scores

---
## **[D] Properties of the principal component scores**

The principal component scores are designed such that they have a number of different properties:

- Principal component scores are **centered** (have zero mean)

In [ ]:
pc_scores.mean().round(6)

- Principal component scores are **sorted in descending order of variance**

In [ ]:
pc_scores.var()

- Principal component scores are **uncorrelated**
 - By design, they are **efficient** in the sense that PC1 does not contain any information about PC2

In [ ]:
pc_scores.cov().round(6)

- Capture all the variation in the two length variables
 - Note: This only holds if we keep **all** principal components
 - That is, if $L = P$ (we will get to this later)

In [ ]:
np.sum(X_only_length.var())

In [ ]:
np.sum(pc_scores.var())

In [ ]:
assert np.isclose(np.sum(X_only_length.var()), np.sum(pc_scores.var()))

### Computing principal component scores

Each principal component score is computed as follows.

For each observation and each principal component: The score for the principal component is obtained by taking the observation's values of the **original variables** and **multiplying** them with the **coefficients** of the principal component direction.

> $\text{score}_{i,d} = x_{i,1} \text{direction}_{d,1} + x_{i,2} \text{direction}_{d, 2}.$

For example, consider the 7th observation (at index 6) and the 2nd principal component (at index 1):

> $\text{score}_{7,2} = x_{7,1} \text{direction}_{2,1} + x_{7,2} \text{direction}_{2, 2}.$

The principal component score:

In [ ]:
np.sum(X_only_length.iloc[6] * pc_directions.iloc[1])

Let's verify that this is equal to score of the 2nd principal component for the 7th observation:

In [ ]:
pc_scores.iloc[6, 1]

The above can be summarized using a **vector-vector** multiplication (dot product) between the vector of values for the 7th observation and the direction vector for the 2nd principal component:

In [ ]:
X_only_length.iloc[6, :] @ pc_directions.iloc[1, :]

Similarly, we can get **all** principal component scores for the 7th observation using a **matrix-vector** multiplication between the **`directions`** matrix for **all** principal components and the vector of values for the 7th observation:

In [ ]:
pc_directions @ X_only_length.iloc[6, :]

Again, let's verify:

In [ ]:
pc_scores.iloc[6]

Finally, we can compute the principal component scores for **all** observations and **all** principal components using a **matrix-matrix** multiplication between the data matrix and the **transposed** directions matrix:

In [ ]:
X_only_length @ pc_directions.T

Again, let's verify:

In [ ]:
pc_scores

### Interpreting the principal component scores

The next step is to provide an interpretation to the principal component scores. For example:

> What does it mean if an observation has a high PC1 score?

We will rephrase this question slightly:

> **Question**: When do we **expect** an observation to have a large positive PC1 score?

To answer this question, we first need to examine the **direction** corresponding to PC1:

In [ ]:
pc_directions.iloc[0]

Note that both variables **positively** contribute to PC1.

We **expect** an observation to have a large positive PC1 score if:
- The value corresponding to petal length is **positive**
- The value corresponding to sepal length is **positive**

However, remember that we have **centered** our data:
- Positive values correspond to **above average** values

This allows us to provide a **more intuitive** interpretation.

Restating the above, we **expect** an observation to have a large positive PC1 score if:
- The value corresponding to petal length (cm) is **above average**
- The value corresponding to sepal length (cm) is **above average**

Also, because the coefficient for petal length is larger (0.919279) than the coefficient for sepal length (0.393606), intuitively petal length will have a **larger influence** on the score for PC1.

Let's verify this intuition by creating a scatterplot of the PC1 and PC2 scores.

We have done something similar in the previous lecture, but now we will add **index labels** for each observation to the plot as well:

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=[20, 10])

# Scatterplot between petal length (horizontal) and sepal length (vertical)
ax[0].set_title('Scatter plot between sepal length and petal length')
ax[0].set_xlabel('Sepal length (centered)')
ax[0].set_ylabel('Petal length (centered)')

ax[0].axvline(0)
ax[0].axhline(0)

ax[0] = sns.scatterplot(
    x=X_only_length.loc[:, 'sepal length (cm)'],
    y=X_only_length.loc[:, 'petal length (cm)'],
    ax=ax[0],
)
ax[0].axis('equal')

# Add index label for each observation
for row_idx, (row_x, row_y) in X_only_length.iterrows():
    ax[0].text(row_x, row_y, row_idx)

# Scatterplot between PC1 (horizontal) and PC2 (vertical)
plt.title('Scatter plot between PC1 and PC2')
plt.xlabel('PC1')
plt.ylabel('PC2')

ax[1].axvline(0)
ax[1].axhline(0)

ax[1] = sns.scatterplot(
    x=pc_scores.loc[:, 'PC1'],
    y=pc_scores.loc[:, 'PC2'],
    ax=ax[1],
)
ax[1].axis('equal')

# Add index label for each observation
for row_idx, (row_x, row_y) in pc_scores.iterrows():
    ax[1].text(row_x, row_y, row_idx)

plt.show()

From the two plot it follows that:
- Observations that have an above average sepal length and petal length (for example: observations 118, 122, 105, 131) also have a relatively **large positive** PC1 score

- Observations that have a below average sepal length and petal length (for example: observations 13, 22) also have a relatively **large negative** PC2 score

In other words, if we want to **describe** PC1, we could state that it measures the **length** of a plant in both the sepal and petal dimension, with the caveat that petal length has a larger influence on this score (in other words, it does not represent just the unweighted combination  of sepal and petal length)

---

We could ask a similar question for PC2:

> **Question**: When do we **expect** an observation to have a high PC2 score?

Examining the **direction** corresponding to PC2:

In [ ]:
pc_directions.iloc[1]

Note that **sepal length** has a **negative** contribution to PC2, while **petal length** has a **positive** contribution to PC2.

We **expect** an observation to have a large positive PC2 score if:
- The value corresponding to petal length is **above average** (positive)
- The value corresponding to sepal length is **below average** (negative)

Also, because the coefficient for sepal length is larger in an absolute sense (-0.919279) than the coefficient for petal length (0.393606), intuitively sepal length will have a **larger influence** on the score for PC2.

We can again confirm this intuition by examining the scatter plots above.

---
## [E] PCA on the full Iris data set

Now that we have an understanding of **principal directions** and **principal component scores**, we can move on and apply PCA to the complete Iris data set, containing 4 variables. We will store the results in the `pca_full` object:

In [ ]:
pca_full = PCA()
pca_full.fit(X)

In [ ]:
pc_full_directions = pd.DataFrame(
    pca_full.components_,
    index=['PC' + str(i + 1) for i in range(len(X.columns))],
    columns = X.columns
)
pc_full_scores = pd.DataFrame(
    pca_full.transform(X),
    columns=['PC' + str(i + 1) for i in range(len(X.columns))],
)

Now that we have a larger matrix with PC directions we should spend some time on how to interpret these numbers, which represent **coefficients**.

In [ ]:
pc_full_directions

In thie interpretation we will focus per principal direction, that is, per row:
We will focus on the interpretation per principal direction, i.e. per row:

**Sign of the coefficients per row**
> - If you want to interpret the **relative importance** of a variable for a principal direction, you should ignore the **sign** of the coefficient and only focus on its **absolute value**

**Maximum (absolute) value of the coefficient per row**
> - The largest coefficient that we can find per row is equal to 1 or -1 (again, ignore the sign here)
> - The closer one of the coefficients is to 1 or -1, the more influential that variable is
> - If one of the coefficients is exactly equal to 1 or -1, all the other coefficients for that direction should be equal to 0

**To interpret a principal direction (each row)**
> - Look for variables that have a large coefficient (in absolute sense, so close to 1 or -1), because they have a large influence on the score of the principal component
> - Variables with smaller coefficients closer to 0 should be ignored, or receive less emphasis, as these do not contribute as much to the score for that principal direction

In [ ]:
pc_full_directions

To provide an interpretation for the principal directions above:
> - PC1 is primarily about petal length. A high score for PC1 likely corresponds with a petal length that is above arage. Conversely, a low score for PC1 likely corresponds with a petal length that is below average
> - PC2 describes the size of the sepal. A high score for PC2 likely corresponds with a relatively large sepal (above average), while a low score for PC2 likely corresponds with a relatively small sepal (below average).
> - PC3 is a little more work to interpret. It does not load high on petal length so we can ignore that variable. It places approximately the same weight (in absolute value) on the other 3 variables in the data. Note that both width variables have a positive sign, while the sepal length variable has a negative sign. This implies that the PC3 score seems to correspond to the ratio of general flower width to sepal length. A high value for PC3 implies that a flower likely has an above average width and a below average sepal length. A low value for PC3 implies that a flower likely has a below average width and an above average sepal length

---
## [F] Explained variance by each principal component

### The `explained_variance_` attribute

The `explained_variance_` attribute of the PCA object gives the **eigenvalue** for each principal component:

In [ ]:
eigenvalues = pca_full.explained_variance_
eigenvalues

Each eigenvalue describes **how much variation** is explained by each principal component.

- Remember, the principal components are constructed such that the first principal component explains the most variance, the second principal component the second most variance, and so and so forth
- This also shows from the eigenvalues, as those are sorted in descending order

Let's look at the **first eigenvalue**, which corresponds to the **first principal component**

In [ ]:
eigenvalues[0]

Compare this against the variance of the first principal component score:

In [ ]:
pc_full_scores.loc[:, 'PC1'].var()

Indeed, the first eigenvalue is identical to the variance of the first principal component score.

Similarly, for all eigenvalues and all principal component scores:

In [ ]:
eigenvalues

In [ ]:
pc_full_scores.var()

In fact, the sum of the eigenvalues is equal to the sum of the variance in the data set:

In [ ]:
np.sum(eigenvalues)

and:

In [ ]:
np.sum(X.var())

and naturally also identical to the sum of the variance in the principal component scores:

In [ ]:
np.sum(pc_full_scores.var())

### The `explained_variance_ratio_` attribute

The `explained_variance_ratio_` attribute gives the ratio of total variance explained by each principal component:

In [ ]:
pca_full.explained_variance_ratio_

Each of these numbers tell us the ratio of the **total variation** in the data that is explained by each principal component.

In this case:
- PC1 accounts for approximately 92.46% of all the variation in the data
- PC2 accounts for another 5.31%
- The remaining variation (about 2%) is explained by PC3 and PC4

In other words:
- PC3 and PC4 do not add a lot of information
- The combination of PC1 and PC2 is able to summarize the variation of **all 4 variables in the data** very well

This is what we typically see in PCA solutions. We only need a small number of principal components to summarize a lot of the variation the data, even if the data consists of many (hundreds) of variables.

Note that the `explained_variance_ratio_` adds up to 1:

In [ ]:
np.sum(pca_full.explained_variance_ratio_)

Note that the `explained_variance_ratio_` attribute is just here for convenience, each element is equal to the corresponding eigenvalue divided by its sum:

In [ ]:
eigenvalues / np.sum(eigenvalues)

---
## [G] Dimensionality reduction using PCA

The **number of principal components that we retain** is given by the number $L$. Setting $L < P$ will lead to **dimensionality reduction**.

The **eigenvalues** can directly be used for some (heuristic) tests to determine the number of principal components we would like to retain in our analysis. The intuition here is that we only want to **keep those principal components that explain a lot of variation** in the data.

We will discuss 3 heuristic tests to determine $L$:
- Kaiser's rule
- Scree plot
- Proportion of variance explained



### Kaiser's rule

**Kaiser's rule**: Only keep the principal components that have an eigenvalue larger than 1.
- Only works well on data that has been standardized (such that all variables have a variance of 1)

> Intuition: We only keep principal components that explain the variance of **at least one variable**.



In [ ]:
eigenvalues

In this case, using Kaiser's rule we would keep only the first principal component

### Scree plot

**Scree plot**: Create a **line plot** of the eigenvalues and only retain the principal components **before** the eigenvalues start to level off
> Often, the scree plot has the shape of an **elbow**, sometimes the scree plot is also referred to as the "elbow plot"

> The scree test then corresponds to finding the principal component index that corresponds to the "bend of the elbow". This is the point where the eigenvalues start to level off.

> The scree test then says that we should select all the principal components **before** the bend of the elbow.

In [ ]:
plt.figure(figsize=(10, 10))

plt.title('Scree plot of the eigenvalues')
plt.xlabel('Principal component index')
plt.ylabel('Eigenvalue')

sns.lineplot(
    x=np.arange(len(eigenvalues)) + 1,
    y=eigenvalues,
    marker='o',
    markersize=12,
)
plt.xticks(np.arange(len(eigenvalues)) + 1)
plt.axis('equal')
plt.show()

In the scree plot above, the "bending point of the elbow" can be clearly defined as occuring **at** the second principal component:

> This implies that starting from the second principal components, the eigenvalues level off, and each principal component more or less explains the same amount of variance

In this case, using the scree plot, we would only keep the first principal component.

**Gotcha #1**: You should be aware that the scree plot does not always represent an elbow! In particular this is the case if PCA is not able to summarize information in just a few components

**Gotcha #2**: In this case the Kaiser rule and the scree plot resulted in the same number of selected components. In general, this does not have to be the case!